In [ ]:
import pandas as pd
df = pd.read_excel('data\\GSA_info.xlsx', sheet_name="public_wells")
ps_codes = [i for i in df['PS Code'].unique()]

In [ ]:
import polars as pl
from pathlib import Path
folder_path = Path(r"C:\Users\Denver\Downloads\SWRCB_WQ")

dfs = [
	pl.read_csv(
		file_path,
		# Default is , but the data is tab delimited
		sep="\t",
		# characters that are not valid utf8 will be ignored
		ignore_errors=True,
		# this tries to guess what the data is but we will assume it is all strings
		infer_schema_length=0
		# Assume all data is strings and remove leading and trailing whitespace
		).with_columns(
			pl.all().cast(pl.Utf8).str.strip() , strict=False
			).filter(pl.col('PS Code').str.strip().is_in(ps_codes))
		# do this for all .tab files in folder
		for file_path in folder_path.glob('*.tab')
		]
# combine all the dataframes into one
df = pl.concat(dfs)
# save as parquet
df.write_parquet(folder_path.joinpath("gsa_swrcb_wq.parquet"))